In [ ]:
pip install chromadb

In [1]:
import chromadb

# Get the Chroma Client
chroma_client = chromadb.Client()

In [2]:
# 建立collection
collection = chroma_client.create_collection(name="my_collection")

In [6]:
import pandas as pd
import csv

# 資料集來源：https://www.kaggle.com/datasets/zake7749/pttgossipingcorpus
# 讀取dataset的csv檔案，只取前10000筆資料
dataset_ptt = pd.read_csv('/kaggle/input/ml-hw5-prob2/Gossiping-QA-Dataset-2_0.csv',nrows=10000)

# 查看csv的檔案內容
# print(dataset_ptt.head())
print(dataset_ptt)

# 確認csv的column有哪些
print(dataset_ptt.columns)

                  question                        answer
0     為什麼 聖結石 會被酸而 這群人 不會？  質感 劇本 成員 都差很多好嗎 不要拿腎結石來污辱這群人
1      為什麼慶祝228會被罵可是慶端午不會？               因為屈原不是台灣人，是楚國人。
2             有沒有戰神阿瑞斯的八卦?               爵士就是阿瑞斯 男主角最後死了
3               理論與實務最脫節的系       哪個系不脫節...你問最不脫節的簡單多了...
4            為什麼PTT這麼多人看棒球                 肥宅才看棒球　系壘一堆胖子
...                    ...                           ...
9995       要聽國語請按1 要聽台語請按2                      要聽法文 請按屄
9996        在麥當勞單點大可是不是笨蛋？      要單買大可當然要使用甜心卡了!同樣價格,買A送B
9997                繳納兩份勞保             只有生病才會回台灣的人會感謝你QQ
9998           這蛋糕鄉民願意掏多少？                  幹日本高級甜點都沒這價錢
9999                   赤い糸               紅線不是結衣演的吧 他只有翻唱

[10000 rows x 2 columns]
Index(['question', 'answer'], dtype='object')


In [7]:
'''
# 測試直接添加資料到collection
collection.add(
    documents=["為什麼 聖結石 會被酸而 這群人 不會？", "質感 劇本 成員 都差很多好嗎 不要拿腎結石來污辱這群人","為什麼慶祝228會被罵可是慶端午不會？","因為屈原不是台灣人，是楚國人。"],
    metadatas=[{"source": "my_source"}, {"source": "my_source"},{"source": "my_source"},{"source": "my_source"}],
    ids=["id1", "id2","id3","id4"]
)

'''

# 將資料集中的問題(Q)和相應的答案(A)添加到collection中
for index, row in dataset_ptt.iterrows():
    question = row['question']
    answer = row['answer']
    
    # 將問題和答案作為文檔添加到集合中
    collection.add(
        documents=[question, answer],
        
        # 添加文檔的元數據，以便識別問題和答案
        metadatas=[{"type": "question"}, {"type": "answer"}],
        
        # 添加後綴作為文檔的ID，方便後續查詢時識別文檔
        ids=[str(index) + "_question", str(index) + "_answer"]
    )

print("Documents added to ChromaDB collection successfully!")

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 64.0MiB/s]


Documents added to ChromaDB collection successfully!


In [15]:
# 查詢文字
question_text = "統神"

# 提供要查詢的問題文本
results = collection.query(
    query_texts=[question_text],
    
    # 只返回最相關的一個文檔，即相對應的答案
    n_results=1
)

# 取得文檔的ids
document_id = results['ids'][0][0]

# 將問題後綴替換為answer
document_id = document_id.replace("question", "answer")

# 印出ids
#print("Document ID:", document_id)

# 取得答案文本
answer_document = collection.get(document_id)

# 印出問題和答案
print("Question：" + question_text)
print("Answer：" + answer_document['documents'][0])

Question：統神
Answer：神 鬼鬼臭酸宅
